# Face Detection from scracth

Importing libraries to gather images dataset

In [7]:
import os
import time
import uuid
import cv2

In [8]:
Image_path=r"C:\Users\Lenovo\Desktop\NEPR\data\images"
number_images=90

Capturing sample images

In [9]:
cap=cv2.VideoCapture(0)
for i in range(number_images):
    print('Collecting images-{}'.format(i))
    ret,frame=cap.read()
    if ret==True:
        imgname=os.path.join(Image_path,f'{str(uuid.uuid1())}.jpg')
        cv2.imwrite(imgname, frame)
        cv2.imshow('frame',frame)
    time.sleep(0.5)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

# Labelling the images

In [10]:
!labelme

Making new folders to put images

In [11]:
newpath = r'C:\Users\Lenovo\Desktop\NEPR\data\train' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
newpath = r'C:\Users\Lenovo\Desktop\NEPR\data\test' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
newpath = r'C:\Users\Lenovo\Desktop\NEPR\data\val' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [12]:
newpath = r'C:\Users\Lenovo\Desktop\NEPR\data\train\images' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
newpath = r'C:\Users\Lenovo\Desktop\NEPR\data\train\labels' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
newpath = r'C:\Users\Lenovo\Desktop\NEPR\data\test\images' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
newpath = r'C:\Users\Lenovo\Desktop\NEPR\data\val\images' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
newpath = r'C:\Users\Lenovo\Desktop\NEPR\data\test\labels' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
newpath = r'C:\Users\Lenovo\Desktop\NEPR\data\val\labels' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

# Creating a tensorflow tensor(dataset) of all images and loading it

In [13]:
import tensorflow as tf
import json
import numpy as np
import matplotlib.pyplot as plt

In [14]:
import tensorflow.keras

To create a dataset of all files matching a pattern, use tf.data.Dataset.list_files

In [15]:
images=tf.data.Dataset.list_files(r'C:\Users\Lenovo\Desktop\NEPR\data\images\*.jpg',shuffle=False) 

This will return the first element of the dataset as a NumPy array.

In [16]:
images.as_numpy_iterator().next()

b'C:\\Users\\Lenovo\\Desktop\\NEPR\\data\\images\\53ec8754-c30d-11ed-8b4d-8c8caa3dd0da.jpg'

Function to load images

In [17]:
def load_image(x):
    byte_img=tf.io.read_file(x) # reads content of the file
    img=tf.io.decode_jpeg(byte_img) # Decode a JPEG-encoded image to a uint8 tensor.
    return img

In [18]:
images=images.map(load_image)

Getting a list of all the images and randomly alloting them to the train, test and validation.

In [19]:
import os
import random
import shutil

# Set up source and destination directories
src_dir = Image_path
train_dir = r'C:\Users\Lenovo\Desktop\NEPR\data\train\images'
test_dir = r'C:\Users\Lenovo\Desktop\NEPR\data\test\images'
val_dir = r'C:\Users\Lenovo\Desktop\NEPR\data\val\images'

# Create destination directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Get a list of all image filenames in the source directory
all_images = os.listdir(src_dir)
image_count = len(all_images)

# Shuffle the list of image filenames
random.shuffle(all_images)

# Move images to train, test, and val directories
for i, image in enumerate(all_images):
    if i < 62:
        dest_dir = train_dir
    elif i < 76:
        dest_dir = test_dir
    else:
        dest_dir = val_dir
        
    shutil.move(os.path.join(src_dir, image), os.path.join(dest_dir, image))

In [20]:
x=r'C:\Users\Lenovo\Desktop\NEPR'
for folder in ['train','test','val']:
    for file in os.listdir(os.path.join(x,'data',folder,'images')):
        filename=file.split('.')[0]+'.json'
        existing_filepath=os.path.join(x,'data','labels',filename)
        if os.path.exists(existing_filepath):
            new_filepath=os.path.join(x,'data',folder,'labels',filename)
            os.replace(existing_filepath,new_filepath)

# Data Augmentation using albumentations library

In [21]:
import albumentations as alb

In [22]:
augmentor = alb.Compose([alb.RandomCrop(width=450, height=450), 
                         alb.HorizontalFlip(p=0.5), 
                         alb.RandomBrightnessContrast(p=0.2),
                         alb.RandomGamma(p=0.2), 
                         alb.RGBShift(p=0.2), 
                         alb.VerticalFlip(p=0.5)], 
                       bbox_params=alb.BboxParams(format='albumentations', 
                                                  label_fields=['class_labels']))

In [23]:
x=r'C:\Users\Lenovo\Desktop\NEPR\data\aug_data'

In [24]:
newpath = r'C:\Users\Lenovo\Desktop\NEPR\data\aug_data\train' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
newpath = r'C:\Users\Lenovo\Desktop\NEPR\data\aug_data\train\images' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
newpath = r'C:\Users\Lenovo\Desktop\NEPR\data\aug_data\train\labels' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
newpath = r'C:\Users\Lenovo\Desktop\NEPR\data\aug_data\test' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
newpath = r'C:\Users\Lenovo\Desktop\NEPR\data\aug_data\test\images' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
newpath = r'C:\Users\Lenovo\Desktop\NEPR\data\aug_data\test\labels' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
newpath = r'C:\Users\Lenovo\Desktop\NEPR\data\aug_data\val' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
newpath = r'C:\Users\Lenovo\Desktop\NEPR\data\aug_data\val\images' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
newpath = r'C:\Users\Lenovo\Desktop\NEPR\data\aug_data\val\labels' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [25]:
x=r'C:\Users\Lenovo\Desktop\NEPR'

In [26]:
for partition in ['train','test','val']: 
    for image in os.listdir(os.path.join(x,'data', partition, 'images')):
        img = cv2.imread(os.path.join(x,'data', partition, 'images', image))

        coords = [0,0,0.00001,0.00001]
        label_path = os.path.join(x,'data', partition, 'labels', f'{image.split(".")[0]}.json')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label = json.load(f)

            coords[0] = label['shapes'][0]['points'][0][0]
            coords[1] = label['shapes'][0]['points'][0][1]
            coords[2] = label['shapes'][0]['points'][1][0]
            coords[3] = label['shapes'][0]['points'][1][1]
            coords = list(np.divide(coords, [640,480,640,480]))

        try: 
            for y in range(60):
                augmented = augmentor(image=img, bboxes=[coords], class_labels=['face'])
                cv2.imwrite(os.path.join(x,'data','aug_data', partition, 'images', f'{image.split(".")[0]}.{y}.jpg'), augmented['image'])

                annotation = {}
                annotation['image'] = image

                if os.path.exists(label_path):
                    if len(augmented['bboxes']) == 0: 
                        annotation['bbox'] = [0,0,0,0]
                        annotation['class'] = 0 
                    else: 
                        annotation['bbox'] = augmented['bboxes'][0]
                        annotation['class'] = 1
                else: 
                    annotation['bbox'] = [0,0,0,0]
                    annotation['class'] = 0 


                with open(os.path.join(x,'data','aug_data', partition, 'labels', f'{image.split(".")[0]}.{y}.json'), 'w') as f:
                    json.dump(annotation, f)

        except Exception as e:
            print(e)

# Images loading

In [27]:
train_images=tf.data.Dataset.list_files(r'C:\Users\Lenovo\Desktop\NEPR\data\aug_data\train\images\*.jpg',shuffle=False)
train_images=train_images.map(load_image)
train_images=train_images.map(lambda x: tf.image.resize(x,(120,120)))
train_images=train_images.map(lambda x: x/255)

test_images=tf.data.Dataset.list_files(r'C:\Users\Lenovo\Desktop\NEPR\data\aug_data\test\images\*.jpg',shuffle=False)
test_images=test_images.map(load_image)
test_images=test_images.map(lambda x: tf.image.resize(x,(120,120)))
test_images=test_images.map(lambda x: x/255)

val_images=tf.data.Dataset.list_files(r'C:\Users\Lenovo\Desktop\NEPR\data\aug_data\val\images\*.jpg',shuffle=False)
val_images=val_images.map(load_image)
val_images=val_images.map(lambda x: tf.image.resize(x,(120,120)))
val_images=val_images.map(lambda x: x/255)

# Label Loading

In [28]:
def load_labels(label_path):
    with open(label_path.numpy(),'r',encoding="utf-8") as f:
        label=json.load(f)
    return [label['class']], label['bbox']

In [29]:
train_labels=tf.data.Dataset.list_files(r'C:\Users\Lenovo\Desktop\NEPR\data\aug_data\train\labels\*.json',shuffle=False)
train_labels=train_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

In [30]:
test_labels=tf.data.Dataset.list_files(r'C:\Users\Lenovo\Desktop\NEPR\data\aug_data\test\labels\*.json',shuffle=False)
test_labels=test_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

In [31]:
val_labels=tf.data.Dataset.list_files(r'C:\Users\Lenovo\Desktop\NEPR\data\aug_data\val\labels\*.json',shuffle=False)
val_labels=val_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

In [32]:
len(train_images), len(train_labels), len(test_images), len(test_labels), len(val_images), len(val_labels)

(3720, 3720, 840, 840, 840, 840)

# Calling the images in batches so that the GPU isn't fully consumed 

In [33]:
train = tf.data.Dataset.zip((train_images, train_labels))
train = train.shuffle(5000)
train = train.batch(8)
train = train.prefetch(4)

test = tf.data.Dataset.zip((test_images, test_labels))
test = test.shuffle(1300)
test = test.batch(8)
test = test.prefetch(4)

val = tf.data.Dataset.zip((val_images, val_labels))
val = val.shuffle(1000)
val = val.batch(8)
val = val.prefetch(4)

# Data Visualization of dataset

In [3]:
data_samples = train.as_numpy_iterator()
res = data_samples.next()
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4): 
    sample_image = res[0][idx]
    sample_coords = res[1][1][idx]
    
    cv2.rectangle(sample_image, 
                  tuple(np.multiply(sample_coords[:2], [120,120]).astype(int)),
                  tuple(np.multiply(sample_coords[2:], [120,120]).astype(int)), 
                        (255,0,0), 2)

    ax[idx].imshow(sample_image)

# Using VGG16 as the backbone, we create a model for object detection 

which detects face as the only object and remove the last layer of the VGG16 to use it only for 1 object detection.

In [35]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16

# Model building

In [36]:
vgg=VGG16(include_top=False)

In [37]:
def build_model():
    input_layer=Input(shape=(120,120,3))
    
    vgg=VGG16(include_top=False)(input_layer)
    f1=GlobalMaxPooling2D()(vgg)
    
    #Classification Model
    class1=Dense(2048,activation='relu')(f1)
    class2=Dense(1,activation='sigmoid')(class1)
    
    #Bounding Box Model
    f2=GlobalMaxPooling2D()(vgg)
    regress1=Dense(2048, activation='relu')(f2)
    regress2=Dense(4,activation='sigmoid')(regress1)
    
    facetracker=Model(inputs=input_layer, outputs=[class2,regress2])
    
    return facetracker

In [38]:
facet=build_model()

In [39]:
facet.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 120, 120, 3) 0                                            
__________________________________________________________________________________________________
vgg16 (Functional)              (None, None, None, 5 14714688    input_2[0][0]                    
__________________________________________________________________________________________________
global_max_pooling2d (GlobalMax (None, 512)          0           vgg16[0][0]                      
__________________________________________________________________________________________________
global_max_pooling2d_1 (GlobalM (None, 512)          0           vgg16[0][0]                      
______________________________________________________________________________________________

In [40]:
X, y = train.as_numpy_iterator().next()

In [41]:
classes, coords = facet.predict(X)

In [42]:
classes, coords

(array([[0.6562486 ],
        [0.55615944],
        [0.60473657],
        [0.5542188 ],
        [0.5566731 ],
        [0.6106665 ],
        [0.640695  ],
        [0.5560473 ]], dtype=float32),
 array([[0.3851599 , 0.55994153, 0.54812646, 0.40848655],
        [0.4544277 , 0.45799834, 0.60939586, 0.5412904 ],
        [0.37409174, 0.597939  , 0.584196  , 0.4253002 ],
        [0.46068138, 0.49632058, 0.5696219 , 0.5194828 ],
        [0.39171106, 0.51848817, 0.6187348 , 0.5366577 ],
        [0.44175845, 0.5965603 , 0.5929734 , 0.5009631 ],
        [0.46395427, 0.6194469 , 0.52898335, 0.45451048],
        [0.4708101 , 0.5963392 , 0.5684964 , 0.48080784]], dtype=float32))

In [43]:
batches_per_epoch=len(train)
lr_decay=(1/0.75-1)/batches_per_epoch

In [44]:
opt=tf.keras.optimizers.Adam(learning_rate=0.0001, decay=lr_decay)

In [45]:
def localization_loss(y_true,y_hat):
    delta_coord=tf.reduce_sum(tf.square(y_true[:,:2]-y_hat[:,:2]))
    
    h_true=y_true[:,3]-y_true[:,1]
    w_true=y_true[:,2]-y_true[:,0]
    
    h_pred=y_hat[:,3]-y_hat[:,1]
    w_pred=y_hat[:,2]-y_hat[:,0]
    
    delta_size=tf.reduce_sum(tf.square(w_true-w_pred)+tf.square(h_true-h_pred))
    
    return delta_coord+delta_size


classloss=tf.keras.losses.BinaryCrossentropy()
regressloss=localization_loss

# Training Pipeline

In [46]:
class FaceTracker(Model):
    def __init__(self, facetracker, **kwargs):
        super().__init__(**kwargs)
        self.model=facetracker
        
    def compile(self, opt, classloss, localizationloss, **kwargs):
        super().compile(**kwargs)
        self.closs=classloss
        self.llos=localizationloss
        self.opt=opt
    
    def train_step(self, batch, **kwargs):
        X,y= batch
        
        with tf.GradientTape() as tape:
            classes, coords=self.model(X, training=True)
            
            batch_classloss=self.closs(y[0], classes)
            batch_localizationloss=self.llos(tf.cast(y[1],tf.float32), coords)
            
            total_loss=batch_localizationloss+0.5*batch_classloss
            
            grad=tape.gradient(total_loss, self.model.trainable_variables)
            
        opt.apply_gradients(zip(grad, self.model.trainable_variables))
        
        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}
    
    def test_step(self, batch, **kwargs):
        
        X,y=batch
        classes, coords=self.model(X, training=True)
            
        batch_classloss=self.closs(y[0], classes)
        batch_localizationloss=self.llos(tf.cast(y[1],tf.float32), coords)
        total_loss=batch_localizationloss+0.5*batch_classloss
        
        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}

    
    def call(self, X, **kwargs):
        return self.model(X, **kwargs)

In [47]:
model=FaceTracker(facet)
model.compile(opt, classloss, regressloss)

In [48]:
logdir='logs'

In [49]:
tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [50]:
hist=model.fit(train, epochs=40, validation_data=val, callbacks=[tensorboard_callback])

Epoch 1/40
465/465 [==============================] - 97s 180ms/step - total_loss: 0.2120 - class_loss: 0.0684 - regress_loss: 0.1778 - val_total_loss: 0.0126 - val_class_loss: 8.8519e-04 - val_regress_loss: 0.0122
Epoch 2/40
465/465 [==============================] - 78s 162ms/step - total_loss: 0.0247 - class_loss: 0.0075 - regress_loss: 0.0210 - val_total_loss: 0.0046 - val_class_loss: 2.5608e-04 - val_regress_loss: 0.0045
Epoch 3/40
465/465 [==============================] - 77s 160ms/step - total_loss: 0.0036 - class_loss: 8.9843e-05 - regress_loss: 0.0035 - val_total_loss: 0.0038 - val_class_loss: 1.2471e-04 - val_regress_loss: 0.0037
Epoch 4/40
465/465 [==============================] - 77s 160ms/step - total_loss: 0.0025 - class_loss: 4.2085e-05 - regress_loss: 0.0025 - val_total_loss: 0.0056 - val_class_loss: 5.7813e-05 - val_regress_loss: 0.0056
Epoch 5/40
465/465 [==============================] - 77s 160ms/step - total_loss: 0.0022 - class_loss: 2.6474e-05 - regress_loss: 0

In [51]:
hist.history

{'total_loss': [0.02023053728044033,
  0.00592361856251955,
  0.0030834004282951355,
  0.0019203294068574905,
  0.0018834942020475864,
  0.0030586475040763617,
  0.0016115113394334912,
  0.0012184057850390673,
  0.0011218368308618665,
  0.001892229076474905,
  0.002018890343606472,
  0.001703345449641347,
  0.0012205207021906972,
  0.001116050872951746,
  0.0012678918428719044,
  0.0005508808535523713,
  0.0012820172123610973,
  0.0008360611973330379,
  0.0007341080927290022,
  0.001308199716731906,
  0.000823320122435689,
  0.0004846368101425469,
  0.0008018279331736267,
  0.00039179850136861205,
  0.000366563763236627,
  0.0003935748536605388,
  0.0007421282352879643,
  0.00048128305934369564,
  0.00020339799812063575,
  0.00046295864740386605,
  0.0001361620961688459,
  0.00026447506388649344,
  0.00029362994246184826,
  0.0002729453844949603,
  0.00029791519045829773,
  0.0003897578571923077,
  8.705060463398695e-05,
  7.389362872345373e-05,
  0.00010105592082254589,
  0.0001601464

# Visualization of prediction on data from test set

In [4]:
test_data = test.as_numpy_iterator()
test_sample = test_data.next()
yhat = facet.predict(test_sample[0])
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4): 
    sample_image = test_sample[0][idx]
    sample_coords = yhat[1][idx]
    
    if yhat[0][idx] > 0.9:
        cv2.rectangle(sample_image, 
                      tuple(np.multiply(sample_coords[:2], [120,120]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [120,120]).astype(int)), 
                            (255,0,0), 2)
    
    ax[idx].imshow(sample_image)

# Saving the model

In [53]:
from tensorflow.keras.models import load_model

In [54]:
facet.save('facedetect.h5')

# Loading the model

In [1]:
from tensorflow.keras.models import load_model

facetracker = load_model('facedetect.h5')
import cv2
import tensorflow as tf
import numpy as np

Using our front camera we can now detect face.

In [4]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _ , frame = cap.read()
    frame = frame[50:500, 50:500,:]
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120,120))
    
    yhat = facetracker.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]
    
    if yhat[0] > 0.5: 
        # Controls the main rectangle
        cv2.rectangle(frame, 
                      tuple(np.multiply(sample_coords[:2], [450,450]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [450,450]).astype(int)), 
                            (255,0,0), 2)
        # Controls the label rectangle
        cv2.rectangle(frame, 
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int), 
                                    [0,-30])),
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                    [80,0])), 
                            (255,0,0), -1)
        
        # Controls the text rendered
        cv2.putText(frame, 'Face', tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                               [0,-5])),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    cv2.imshow('Facetrack', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()